### Setting up notebook 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import math
from IPython.core.display import display, HTML
import d3_lib
from string import Template
import pandas as pd
import json
import random
import pickle
from functions import *

%pylab inline

Populating the interactive namespace from numpy and matplotlib


C:\Users\kelihjoll\Anaconda2\lib\site-packages\IPython\core\magics\pylab.py:161: UserWarning: pylab import has clobbered these variables: ['random']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


### Loading Data 

In [2]:
pickle_in = open("einstaedur_oryrki.pickle","rb")
einstaedur_oryrki = pickle.load(pickle_in)
pickle_in = open("einstaedur_ellilif.pickle","rb")
einstaedur_ellilif = pickle.load(pickle_in)
pickle_in = open("fjolskylda.pickle","rb")
fjolskylda = pickle.load(pickle_in)

### Line Graph with Hover

In [3]:
HTML(d3_lib.set_styles(['line_with_hover']))

In [4]:
data = []
tekjulisti = np.arange(0, 1000000, 1000)
for i in tekjulisti:
    einstaedur_oryrki[1]['tekjur'][0] = i
    tekjur_i_vasa = islendingur(einstaedur_oryrki[1])[0]['pjeng_i_vasa']
    data.append({'x': i, 'y':tekjur_i_vasa})
    
HTML(d3_lib.draw_graph('line_with_hover',{'data': data}))

In [5]:
data = []
tekjulisti = np.arange(0, 1000000, 1000)
for i in tekjulisti:
    einstaedur_ellilif[1]['tekjur'][0] = i
    tekjur_i_vasa = islendingur(einstaedur_ellilif[1])[0]['pjeng_i_vasa']
    data.append({'x': i, 'y':tekjur_i_vasa})
    
HTML(d3_lib.draw_graph('line_with_hover',{'data': data}))

In [6]:
data = []
tekjulisti = np.arange(0, 1000000, 1000)
for i in tekjulisti:
    fjolskylda[1]['tekjur'][0] = i
    tekjur_i_vasa = islendingur(fjolskylda[1])[0]['pjeng_i_vasa']
    data.append({'x': i, 'y':tekjur_i_vasa})
    
HTML(d3_lib.draw_graph('line_with_hover',{'data': data}))

## Bar graph with update

In [ ]:
src="http://labratrevenge.com/d3-tip/javascripts/d3.tip.v0.6.3.js"

In [7]:
HTML('<script src="http://d3js.org/d3.v3.min.js" charset="utf-8"></script>')

#### Graph Config 

In [8]:
html_template = Template('''
<style> $css_text </style>
<div id="graph-div"></div>
<script> $js_text </script>
''')

In [9]:
css_text = '''

.bar {
  fill: steelblue;
}

.bar:hover {
  fill: brown;
}

.axis {
  font: 10px sans-serif;
}

.axis path,
.axis line {
  fill: none;
  stroke: #000;
  shape-rendering: crispEdges;
}

.x.axis path {
  display: none;
}

.d3-tip {
  line-height: 1;
  font-weight: bold;
  padding: 12px;
  background: rgba(0, 0, 0, 0.8);
  color: #fff;
  border-radius: 2px;
}

'''

In [10]:
js_text_template = Template('''

var margin = {top: 20, right: 20, bottom: 30, left: 60},
    width = 650 - margin.left - margin.right,
    height = 300 - margin.top - margin.bottom;

var x = d3.scale.ordinal()
    .rangeRoundBands([0, width], .1);

var y = d3.scale.linear()
    .range([height, 0]);

var xAxis = d3.svg.axis()
    .scale(x)
    .orient("bottom");

var yAxis = d3.svg.axis()
    .scale(y)
    .orient("left");
    
var tip = d3.tip()
  .attr('class', 'd3-tip')
  .offset([-10, 0])
  .html(function(d) {
    return "<strong>Pjengs:</strong> <span style='color:red'>" + formatValue(d.y) + " kr." + "</span>";
  })

var svg = d3.select("#graph-div").append("svg")
    .attr("width", width + margin.left + margin.right)
    .attr("height", height + margin.top + margin.bottom)
  .append("g")
    .attr("transform", "translate(" + margin.left + "," + margin.top + ")");

var formatValue = d3.format(",.0f");

svg.call(tip);

var data = $data ;

  x.domain(data.map(function(d) { return d.letter; }));
  y.domain([0, d3.max(data, function(d) { return d.y; })]);

  svg.append("g")
      .attr("class", "x axis")
      .attr("transform", "translate(0," + height + ")")
      .call(xAxis);

  svg.append("g")
      .attr("class", "y axis")
      .call(yAxis);

  svg.selectAll(".bar")
      .data(data)
    .enter().append("rect")
      .attr("class", "bar")
      .attr("x", function(d) { return x(d.letter); })
      .attr("width", x.rangeBand())
      .attr("y", function(d) { return y(d.y); })
      .attr("height", function(d) { return height - y(d.y); })
      .on('mouseover', tip.show)
      .on('mouseout', tip.hide);

function type(d) {
  d.y = +d.y;
  return d;
}
''')

In [11]:
js_text_template_2 = Template('''

var bars = svg.selectAll(".bar").data($data);
  
bars
  .transition()
    .attr("y", function(d) { return y(d.y); })
    .attr("height", function(d) { return height - y(d.y); });

''')

#### Initial Data, Graph and Update 

In [12]:
# loading data
pickle_in = open("einstaedur_oryrki.pickle","rb")
example_dict = pickle.load(pickle_in)

In [13]:
data =\
    pd.DataFrame({'letter':['Laun e skatt',
                            'Fjármagnstekjur',
                            'Húsnæðisstuðningur',
                            'Barnabætur',
                            'Örorkubætur',
                            'Ráðstöfunartekjur'],
                  'y':[example_dict[0]['tekjur_e_skatt'],
                       example_dict[0]['fjarmagnstekjur_e_skatt'],
                       example_dict[0]['husnaedisstudningur'],
                       example_dict[0]['barnabaetur'],
                       example_dict[0]['ororka_e_skatt'],
                       example_dict[0]['pjeng_i_vasa']]})
data

,letter,y
0,Laun e skatt,260374.400000
1,Fjármagnstekjur,20000.000000
2,Húsnæðisstuðningur,32688.103090
3,Barnabætur,64535.425227
4,Örorkubætur,84137.737526
5,Ráðstöfunartekjur,460335.665842


In [14]:
js_text = js_text_template.substitute({'data': json.dumps(data.to_dict(orient='records'))})
HTML(html_template.substitute({'css_text': css_text, 'js_text': js_text}))

In [15]:
# change 'tekjur' parameter and run to update graph
example_dict[1]['tekjur'][0] = 400000
new_results = islendingur(example_dict[1])[0]

data['y'] = [new_results['tekjur_e_skatt'],
             new_results['fjarmagnstekjur_e_skatt'],
             new_results['husnaedisstudningur'],
             new_results['barnabaetur'],
             new_results['ororka_e_skatt'],
            new_results['pjeng_i_vasa']]
js_text = js_text_template_2.substitute({'data': json.dumps(data.to_dict(orient='records'))})
HTML('<script>' + js_text + '</script>')